In [169]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.OnnxConverter"

Installed Packages Microsoft.ML, 2.0.1 Microsoft.ML.OnnxConverter, 0.20.1

In [170]:
#r "nuget: Jieba.Net.Core"

Installed Packages Jieba.Net.Core, 1.1.0

In [171]:
using System;
using System.IO;
using System.Linq;

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

using JiebaNet;

In [172]:
    public class Question
    {
        [LoadColumn(0)]
        public int Label { get; set; }
        [LoadColumn(1)]
        public string Text { get; set; }
    }

In [173]:
public class JiebaLambdaInput
{
    public string Text { get; set; }
}

public class JiebaLambdaOutput
{
    public string JiebaText { get; set; }
}

public class JiebaLambda
{       
        public static void MyAction(JiebaLambdaInput input, JiebaLambdaOutput output)
        {
            JiebaNet.Segmenter.JiebaSegmenter jiebaSegmenter = new JiebaNet.Segmenter.JiebaSegmenter();
            output.JiebaText = string.Join(" ", jiebaSegmenter.Cut(input.Text));

            Count++;
        }

        static int Count = 0;
}

In [174]:
MLContext mlContext = new MLContext();
var fulldata = mlContext.Data.LoadFromTextFile<Question>("./data/qa.csv", separatorChar: ',', hasHeader: false);

In [175]:
var trainTestData = mlContext.Data.TrainTestSplit(fulldata, testFraction: 0.2);
var trainData = trainTestData.TrainSet;
var testData = trainTestData.TestSet;

In [176]:
trainData

Microsoft.ML.Transforms.ColumnSelectingTransformer+SelectColumnsDataTransform CanShuffle False Source Microsoft.ML.Transforms.RangeFilter OutputSchema index value 0 Label: Int32 Name Label Index 0 IsHidden False Type Int32 RawType System.Int32 Annotations Schema (empty) 1 Text: String Name Text Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) 2 SplitColumn: Single {IsNormalized} Name SplitColumn Index 2 IsHidden False Type Single RawType System.Single Annotations IsNormalized Schema index value 0 IsNormalized: Boolean Source Microsoft.ML.Transforms.GenerateNumberTransform OutputSchema index value 0 Label: Int32 Name Label Index 0 IsHidden False Type Int32 RawType System.Int32 Annotations Schema [ ] 1 Text: String Name Text Index 1 IsHidden False Type String RawType System.ReadOnlyMemory`1[System.Char] Annotations Schema [ ] 2 SplitColumn: Single {IsNormalized} Name SplitColumn Index 2 IsHidden False Type Single RawType System.Single Annotations IsNormalized Schema [ IsNormalized: Boolean ] CanShuffle False Source Microsoft.ML.Data.TextLoader+BoundLoader CanShuffle False Schema index value 0 Label: Int32 Name Label Index 0 IsHidden False Type Int32 Annotations 1 Text: String Name Text Index 1 IsHidden False Type String Annotations CanShuffle False InputSchema index value 0 Label: Int32 Name Label Index 0 IsHidden False Type Int32 RawType System.Int32 Annotations Schema (empty) 1 Text: String Name Text Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty) 2 SplitColumn: Single {IsNormalized} Name SplitColumn Index 2 IsHidden False Type Single RawType System.Single Annotations IsNormalized Schema index value 0 IsNormalized: Boolean Name IsNormalized Index 0 IsHidden False Type Boolean Annotations OutputSchema index value 0 Label: Int32 Name Label Index 0 IsHidden False Type Int32 RawType System.Int32 Annotations Schema (empty) 1 Text: String Name Text Index 1 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema (empty)

In [177]:

var trainingPipeline = mlContext.Transforms.CustomMapping<JiebaLambdaInput, JiebaLambdaOutput>(mapAction: JiebaLambda.MyAction, contractName: "JiebaLambda")
               .Append(mlContext.Transforms.Text.FeaturizeText("Features", "JiebaText"))
               .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "LabelKey", inputColumnName: "Label"))
               .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy("LabelKey", "Features"))
               .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));;

                

In [178]:
ITransformer trainedModel = trainingPipeline.Fit(trainData);

In [179]:
    public class PredictionResult : Question
    {
        
        public string JiebaText { get; set; }
        public float[] Features { get; set; }

        public int PredictedLabel;
        public float[] Score;
        public float Probability;
        public void PrintToConsole()
        {
            Console.WriteLine($"JiebaText={JiebaText}");
            Console.WriteLine($"PredictedLabel:{PredictedLabel},Score:{Score},Probability:{Probability}");
            Console.WriteLine($"TextFeatures Length:{Features.Length}");
            if (Features != null)
            {
                foreach (var f in Features)
                {
                    Console.Write($"{f},");
                }
                Console.WriteLine();
            }
            Console.WriteLine();
        }
    }

In [180]:
var predEngine = mlContext.Model.CreatePredictionEngine<Question, PredictionResult>(trainedModel);

In [181]:

Question sampleStatement1 = new Question { Text = "气温多少" };
var predictionresult1 = predEngine.Predict(sampleStatement1);
Console.WriteLine($"{sampleStatement1.Text}:{predictionresult1.PredictedLabel}");

气温多少:1


In [182]:
Question sampleStatement2 = new Question { Text = "什么是新能源车" };
var predictionresult2 = predEngine.Predict(sampleStatement2);
Console.WriteLine($"{sampleStatement2.Text}:{predictionresult2.PredictedLabel}");

什么是新能源车:0


In [183]:
Question sampleStatement2 = new Question { Text = "课程讲什么" };
var predictionresult2 = predEngine.Predict(sampleStatement2);
Console.WriteLine($"{sampleStatement2.Text}:{predictionresult2.PredictedLabel}");

课程讲什么:0
